## Imports

In [1]:
import pyspark.sql.functions as F
from dhzlib import DhzLib

## Teste de Conexão - Banco MySQL

In [2]:
dhz = DhzLib()
dhz.test_connection_mysql()

Connected to MySQL


## Lendo arquivos CSV e escrevendo dados no Banco MySQL

In [ ]:
df = dhz.spark.read.csv('../data/*.csv', header=True, inferSchema=True)
dhz.write_mysql(df, 'raw_capital_bikeshare')
df.count()

## Criando tabela na camada trusted

In [3]:
raw_df = dhz.read_mysql('raw_capital_bikeshare')

Data read from MySQL


In [4]:
trusted_df = raw_df.select(
    F.col('ride_id').alias('ride_id'),
    F.col('rideable_type').alias('rideable_type'),
    F.col('started_at').cast('timestamp').alias('started_at'),
    F.col('ended_at').cast('timestamp').alias('ended_at'),
    F.col('start_station_name').alias('start_station_name'),
    F.col('start_station_id').cast('int').alias('start_station_id'),
    F.col('end_station_name').alias('end_station_name'),
    F.col('end_station_id').cast('int').alias('end_station_id')
)

In [5]:
dhz.write_mysql(trusted_df, 'tr_capital_bikeshare')

## RASCUNHO

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# spark = SparkSession.builder \
#         .appName('Connecting to MySQL') \
#         .getOrCreate()

In [ ]:

url = os.getenv("MYSQL_URL")
mysql_properties = {
    'user': os.getenv("MYSQL_USER"),
    'password': os.getenv("MYSQL_PASSWORD")
}

In [ ]:
teste = spark.read.csv('../data/*.csv', header=True)

In [ ]:
teste.count()